## U-Net training for ISBI challenge

Training and evaluation over the ISBI 2012 dataset of the U-Net model created with PyEDDL library.

**Francisco Javier Blázquez Martínez** \
**francisco.blazquezmartinez@epfl.ch**

Double degree in Mathematics-Computer Engineering

Complutense University of Madrid, Spain \
École Politechnique Fédérale de Lausanne, Switzerland

DeepHealth project

In [2]:
import pyeddl.eddl as eddl
from pyeddl.tensor import Tensor

In [3]:
# Set the following variable to the folder containing the ISBI dataset
# TODO: Submit the general version <ISBI_PATH = None>
ISBI_PATH = "/home/francisco/Documents/Universidad/5_Carrera/TFG_Computer_Science/datasets/isbi/"

#### DRIVE Dataset download

In [ ]:
# http://brainiac2.mit.edu/isbi_challenge/

#### DRIVE Data Augmentation

In [ ]:
# TODO

#### DRIVE Data Preprocessing

In [4]:
# WARNING! Run only once, new tensor files (.bin) are created in the dataset folder

# Resizing implies a loss of accuracy but it's convenient if we have problems with
# the memory usage during training

In [ ]:
from PIL import Image, ImageDraw

In [ ]:
train_imgs = [ISBI_PATH + 'train/image/'+str(i)+'.png' for i in range(30)]
train_masks= [ISBI_PATH + 'train/label/'+str(i)+'.png' for i in range(30)]

In [ ]:
for i in range(30):
    image_input  = ISBI_PATH + 'train/image/'     + str(i) + '.png'
    mask_input   = ISBI_PATH + 'train/label/'     + str(i) + '.png'
    image_output = ISBI_PATH + 'train/image_lq/' + str(i) + '.png'
    mask_output  = ISBI_PATH + 'train/label_lq/' + str(i) + '.png'
    
    img = Image.open(image_input)
    msk = Image.open(mask_input)
    
    img = img.resize((img.size[0]//2, img.size[1]//2))
    msk = msk.resize((msk.size[0]//2, msk.size[1]//2))
    
    img = img.save(image_output)
    msk = msk.save(mask_output)

#### DRIVE Preprocessed data visualization

In [ ]:
import numpy   as np
import matplotlib.pyplot as plt
import random

In [ ]:
tensor_x = Tensor.load(DRIVE_PATH + "drive_trX_preprocessed.bin")
tensor_y = Tensor.load(DRIVE_PATH + "drive_trY_preprocessed.bin")

In [ ]:
fig = plt.figure(figsize=(16, 10))
rows = 2
columns = 4

for i in range(columns):
    idx = random.randint(0, tensor_x.shape[1])
    
    img_np = tensor_x.select([str(i)])
    msk_np = tensor_y.select([str(i)])
    
    # Remove batches axis (and number of channels for the mask)
    img_np.reshape_(img_np.shape[1:])
    msk_np.reshape_(msk_np.shape[2:])
    
    # Prepare to Matplotlib visualization
    img_np = img_np.getdata().astype(np.int32).T
    msk_np = msk_np.getdata().astype(np.int32).T

    # Size check
    #print(img0_np.shape)
    #print(msk0_np.shape)

    fig.add_subplot(rows, columns, i+1)
    plt.imshow(img_np)
    plt.axis('off')
    
    fig.add_subplot(rows, columns, i+1+columns)
    plt.imshow(msk_np) # set cmap="gray" to display the image in B&W
    plt.axis('off')

#### U-Net model train

In [ ]:
# WARNING! Heavy computation, run preferably on the server!

In [ ]:
#from unet import unet

In [ ]:
EPOCHS        = 10
BATCH_SIZE    = 8
NUM_BATCHES   = 50
GPU           = True
MEM           = "full_mem"
LOSS_FUNCTION = "mse"
METRICS       = "mse"
LEARNING_RATE = 0.00001

In [ ]:
in_ = eddl.Input([3, 512, 512])
net = unet(in_)

# TODO: Should we return the model created or the output layer?
#out = unet(in_) 
#net = eddl.Model([in_], [out])

eddl.build(
    net,
    eddl.adam(LEARNING_RATE),  # Optimizer
    [LOSS_FUNCTION],           # Losses
    [METRICS],                 # Metrics
    eddl.CS_GPU(mem=MEM) if GPU else eddl.CS_CPU(mem=MEM)
)

eddl.summary(net)

In [ ]:
x_train = Tensor.load(DRIVE_PATH+"drive_trX_preprocessed.bin")
x_train.div_(255.0)
x_train.info()

y_train = Tensor.load(DRIVE_PATH+"drive_trY_preprocessed.bin")
y_train.div_(255.0)
y_train.info()

xbatch = Tensor([BATCH_SIZE, 3, 512, 512])
ybatch = Tensor([BATCH_SIZE, 1, 512, 512])

In [ ]:
for i in range(EPOCHS):
    print("\nEpoch %d/%d" % (i + 1, EPOCHS))
    eddl.reset_loss(net)
    for j in range(NUM_BATCHES):
        eddl.next_batch([x_train, y_train], [xbatch, ybatch])
        eddl.train_batch(net, [xbatch_da], [ybatch_da])
        eddl.print_loss(net, j)
        
        #if i == EPOCHS - 1:
        #    yout = eddl.getOutput(out).select(["0"])
        #    yout.save("./out_%d.jpg" % j)
        #print()

#### U-Net model save

In [ ]:
eddl.save("models/unet_drive.bin")

#### U-Net model load

In [ ]:
#from unet import unet

In [ ]:
GPU           = True
MEM           = "full_mem"
LOSS_FUNCTION = "mse"
METRICS       = "mse"
LEARNING_RATE = 0.00001

In [ ]:
in_ = eddl.Input([3, 512, 512])
net = unet(in_)

# TODO: Should we return the model created or the output layer?
#out = unet(in_) 
#net = eddl.Model([in_], [out])

eddl.build(
    net,
    eddl.adam(LEARNING_RATE),  # Optimizer
    [LOSS_FUNCTION],           # Losses
    [METRICS],                 # Metrics
    eddl.CS_GPU(mem=MEM) if GPU else eddl.CS_CPU(mem=MEM)
)

eddl.summary(net)

In [ ]:
eddl.load(net, "models/unet_drive.bin")

#### U-Net model visual analysis

In [ ]:
import numpy   as np
import matplotlib.pyplot as plt
import random

In [ ]:
tensor_x = Tensor.load(DRIVE_PATH + "drive_trX_preprocessed.bin")
tensor_y = Tensor.load(DRIVE_PATH + "drive_trY_preprocessed.bin")

In [ ]:
fig = plt.figure(figsize=(24, 10))
rows = 3
columns = 6

for i in range(columns):
    idx = random.randint(0, tensor_x.shape[1])
    
    img_np = tensor_x.select([str(i)])
    msk_np = tensor_y.select([str(i)])
    
    # Remove batches axis (and number of channels for the mask)
    img_np.reshape_(img_np.shape[1:])
    msk_np.reshape_(msk_np.shape[2:])
    
    # Prepare to Matplotlib visualization
    img_np = img_np.getdata().astype(np.int32).T
    msk_np = msk_np.getdata().astype(np.int32).T

    # Size check
    #print(img0_np.shape)
    #print(msk0_np.shape)

    fig.add_subplot(rows, columns, i+1)
    plt.imshow(img_np)
    plt.axis('off')
    
    fig.add_subplot(rows, columns, i+1+columns)
    plt.imshow(msk_np) # set cmap="gray" to display the image in B&W
    plt.axis('off')
    
    # TODO: Here we should take the mask that our model gives!
    fig.add_subplot(rows, columns, i+1+columns*2)
    plt.imshow(msk_np) # set cmap="gray" to display the image in B&W
    plt.axis('off')

#### U-Net model evaluation

In [ ]:
# TODO: Keep some images out of the train set for evaluation!

#### References
http://brainiac2.mit.edu/isbi_challenge/ \
https://paperswithcode.com/sota/medical-image-segmentation-on-isbi-2012-em \
https://github.com/alexklibisz/isbi-2012 \
https://www.frontiersin.org/articles/10.3389/fnana.2015.00142/full